Part1

In [1]:
import cv2
import numpy as np

# 加载图像
m1 = cv2.imread('./test/m1.jpg')
m2 = cv2.imread('./test/m2.jpg')
m3 = cv2.imread('./test/m3.jpg')

# 创建SIFT检测器
sift = cv2.SIFT_create()

def align_images(im1, im2):
    # 转换为灰度图像
    gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # 检测特征点和描述符
    keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

    # 创建BFMatcher对象
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # 匹配描述符
    matches = bf.match(descriptors1, descriptors2)

    # 根据距离排序
    matches = sorted(matches, key=lambda x: x.distance)

    # 提取匹配点坐标
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # 计算单应性矩阵
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # 使用单应性矩阵变换图像
    height, width, channels = im2.shape
    im1_aligned = cv2.warpPerspective(im1, h, (width, height))

    return im1_aligned

# 对齐图像
m1_aligned = align_images(m1, m2)
m2_aligned = m2 # m2 is the reference image
m3_aligned = align_images(m3, m2)

# 保存对齐后的图像
cv2.imwrite('./output/m1_aligned.jpg', m1_aligned)
cv2.imwrite('./output/m2_aligned.jpg', m2_aligned)
cv2.imwrite('./output/m3_aligned.jpg', m3_aligned)

# 分离和保存每个通道
for i, aligned_image in enumerate([m1_aligned, m2_aligned, m3_aligned], start=1):
    zeros = np.zeros_like(aligned_image[:, :, 0])
    for j, channel in enumerate(['b', 'g', 'r']):
        color_image = np.stack([
            aligned_image[:, :, j] if k == j else zeros for k in range(3)
        ], axis=-1)
        cv2.imwrite(f'./output/m{i}-{channel}.jpg', color_image)

Part2

In [2]:
# 分离每个图像的通道
def split_channels(image):
    r, g, b = cv2.split(image)
    return g, b, r  # 注意OpenCV加载图像的顺序是BGR，所以这里需要调整顺序

# 提取通道灰度值
Image1_g = cv2.imread('./test/1.jpg', cv2.IMREAD_GRAYSCALE)
Image1_y = cv2.imread('./test/2.jpg', cv2.IMREAD_GRAYSCALE)
Image1_r = cv2.imread('./test/3.jpg', cv2.IMREAD_GRAYSCALE)

Image2_g = cv2.imread('./test/4.jpg', cv2.IMREAD_GRAYSCALE)
Image2_y = cv2.imread('./test/5.jpg', cv2.IMREAD_GRAYSCALE)
Image2_r = cv2.imread('./test/6.jpg', cv2.IMREAD_GRAYSCALE)

Image3_g = cv2.imread('./test/7.jpg', cv2.IMREAD_GRAYSCALE)
Image3_y = cv2.imread('./test/8.jpg', cv2.IMREAD_GRAYSCALE)
Image3_r = cv2.imread('./test/9.jpg', cv2.IMREAD_GRAYSCALE)


# 获取图像的尺寸
height, width = Image1_g.shape

# 创建一个空矩阵用于存储新的灰度值
new_gray_values = np.zeros((height, width), dtype=np.uint8)

# 遍历每个像素坐标
for i in range(height):
    for j in range(width):
        # 根据条件计算新的灰度值
        if Image1_g[i, j] > 50 and Image1_y[i, j] < 30 and Image1_r[i, j] > 100:
            new_gray_values[i, j] = 90
        elif Image2_g[i, j] > 50 and Image2_y[i, j] < 30 and Image2_r[i, j] > 100:
            new_gray_values[i, j] = 100
        elif Image3_g[i, j] > 250 and Image3_y[i, j] < 30 and Image3_r[i, j] > 100:
            new_gray_values[i, j] = 255
        else:
            new_gray_values[i, j] = 0
# 将新的灰度值矩阵转换为图像
cv2.imwrite('./output/gray_image.jpg', new_gray_values)

True

额外代码：输出图片全黑原因，没有符合要求的坐标点，以下代码进行了手动查找，发现没有这样的点，因此输出图片全黑

In [3]:
nonzero_coords = np.nonzero(Image1_g)
# 打印非零元素的坐标
for coord in zip(*nonzero_coords):
    if Image1_y[coord[0]][coord[1]] < 30 and Image1_r[coord[0]][coord[1]] > 100 and Image1_g[coord[0]][coord[1]] > 50:
        print(f"符合Image1要求的坐标 {coord}")
        print(Image1_y[coord[0]][coord[1]])
print("Image1筛选结束")

nonzero_coords = np.nonzero(Image2_g)
# 打印非零元素的坐标
for coord in zip(*nonzero_coords):
    if Image2_y[coord[0]][coord[1]] < 30 and Image2_r[coord[0]][coord[1]] > 100 and Image2_g[coord[0]][coord[1]] > 50:
        print(f"符合Image2要求的坐标 {coord}")
        print(Image2_y[coord[0]][coord[1]])
print("Image2筛选结束")

nonzero_coords = np.nonzero(Image3_g)
# 打印非零元素的坐标
for coord in zip(*nonzero_coords):
    if Image3_y[coord[0]][coord[1]] < 30 and Image3_r[coord[0]][coord[1]] > 100 and Image3_g[coord[0]][coord[1]] > 50:
        print(f"符合Image3要求的坐标 {coord}")
        print(Image3_y[coord[0]][coord[1]])
print("Image3筛选结束")

Image1筛选结束
Image2筛选结束
Image3筛选结束
